### Finetuning hyperparameters of logistic regression ML algorithm

Installing dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')

sklearn.exceptions.ConvergenceWarning('ignore')

#### Data ingestion

In [3]:
bio= pd.read_csv('healthcare_data.csv')
bio.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
bio.shape

(569, 33)

In [5]:
bio.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [6]:
bio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [7]:
bio.diagnosis.unique()

array(['M', 'B'], dtype=object)

#### Brief description of the data
* Total 569 records of patients and 33 columns
* id is a unique column that does not contribute to prediction
* unnamed 32 is a column with no value in it and hence cannot contribute
* diagnosis is an object type feature with only two values: M and B
* diagnosis is a target variable with two categories: M and B
* remaining all the columns are float type and act as predictor variables

#### Basic Preprocessing

In [8]:
# remove features 'id' and 'Unnamed: 32' since they do not help in predicting
bio=bio.drop(['id','Unnamed: 32'],axis=1)

In [9]:
# Encoding target variable 'diagnosis'
bio['diagnosis']=bio['diagnosis'].map({'M':0,'B':1})

# Encoding: a preprocessing technique used to convert categorical variables
# to number codes.

In [10]:
bio.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [11]:
bio.shape

(569, 31)

#### Above dataset is finally transfromed with all Numerical values,
 * hence all numerical variables must be scaled

#### Splitting the dataset into X and y

In [13]:
X=bio.drop(['diagnosis'],axis=1)
Y=bio['diagnosis']

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,train_size=0.8,random_state=32)

### Logistic Regression is a distence based model, hence all numerical features must be scaled.

In [15]:
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

#### Basic model training and inferencing

##### Instentiation and training

In [16]:
lr=LogisticRegression()
model=lr.fit(X_train,y_train)

##### inferencing

In [17]:
y_pred=model.predict(X_test)

##### Evaluation

In [18]:
print(accuracy_score(y_test,y_pred))

0.9912280701754386


#### Hyperparameter Finetuning
##### Hyperparameter used:
* Panelty: L1,L2 and elasticNet. by default panelty is L2.
* C: regularization parameter. This depicts the complexity of the model.
* Solver: Optimize problems can be solved by different solvers.
* max_iters: in how many steps the global minima is reached.

In [28]:
param_grid=[
    {'penalty':['l1','l2','elasticnet'],
     'C':np.logspace(-4,4,20),
     'solver':['lbfgs','newton-cg','liblinear'],
     'max_iter':[500,1000,1500]
}]
3*20*3*3

540

In [29]:
np.logspace(-4,4,20)  # .0001 to 10000 
# it will generate 20 values equidistence from each other in the range between 10^-4 to 10^4 

array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])

##### Comments
* it is just way to give rangesof values that c can take
* You may also use the simple approach as shown below:
* 'C': [0.0001,.001,.01,.1,1,10,100,1000,10000]

In [30]:
gsc=GridSearchCV(lr,param_grid=param_grid,cv=10,verbose=True,n_jobs=-1)
tuned_model=gsc.fit(X_train,y_train)

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


In [32]:
tuned_model.best_estimator_

LogisticRegression(C=0.615848211066026, max_iter=500, solver='liblinear')

In [34]:
print(f'Accuracy-: {tuned_model.score(X_test,y_test):.3f}')

Accuracy-: 0.991


In [36]:
lr_finetuned=lr.set_params(C=0.615848211066026, max_iter=500,random_state=10)

In [37]:
model_final=lr_finetuned.fit(X_train,y_train)
y_pred=model_final.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9912280701754386


In [38]:
model_final.penalty

'l2'

In [39]:
model_final.coef_

array([[-0.44916702, -0.34263193, -0.44279944, -0.48684082, -0.12377692,
         0.13741954, -0.73097515, -0.74314266,  0.1012043 ,  0.26575039,
        -1.00254556, -0.01363178, -0.56511284, -0.82321554, -0.15458221,
         0.69725742,  0.07841185, -0.25660809,  0.21398656,  0.43877365,
        -0.86967505, -0.99926638, -0.74756783, -0.85306437, -0.67282252,
         0.02635463, -0.71572803, -0.59028552, -0.69931658, -0.35419675]])

#### Trying working with L1

In [40]:
model_l1=LogisticRegression(penalty='l1',C=1,max_iter=10,random_state=10,solver='liblinear')

In [43]:
model_l1_fit=model_l1.fit(X_train,y_train)
y_pred_l1=model_l1.predict(X_test)
accuracy_score(y_test,y_pred_l1)

0.9824561403508771

In [44]:
model_l1_fit.coef_

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.31050001, -1.12438627,  0.        ,  0.        ,
        -2.22121734,  0.        ,  0.        ,  0.        , -0.1204085 ,
         0.7341137 ,  0.        , -0.0807474 ,  0.        ,  0.23224001,
        -1.47232936, -1.47318004, -1.09528432, -2.42525632, -0.76783635,
         0.        , -0.93399423, -0.40264448, -0.44199856,  0.        ]])

#### Observations
* The highest accuracy is obtain after finetuning the model.
* Base model accuracy is less then the fine tuned model accuracy.
* With L1, penalty specific experiment is done just to verify the result.
* The coefficients of l1 model show how many features coefficients are equal
  to zero.
* So L1 does help in feature selection. All non zero coefficient features are consideres
  for final model.